In [2]:
!kaggle competitions download -c rossmann-store-sales


  0%|          | 0.00/55.3k [00:00<?, ?B/s]
100%|##########| 55.3k/55.3k [00:00<00:00, 905kB/s]

  0%|          | 0.00/8.33k [00:00<?, ?B/s]
100%|##########| 8.33k/8.33k [00:00<00:00, 190kB/s]

  0%|          | 0.00/143k [00:00<?, ?B/s]
100%|##########| 143k/143k [00:00<00:00, 1.56MB/s]

  0%|          | 0.00/5.66M [00:00<?, ?B/s]
 18%|#7        | 1.00M/5.66M [00:00<00:02, 2.42MB/s]
 35%|###5      | 2.00M/5.66M [00:00<00:01, 2.43MB/s]
 53%|#####3    | 3.00M/5.66M [00:01<00:01, 2.42MB/s]
 71%|#######   | 4.00M/5.66M [00:01<00:00, 2.48MB/s]
 88%|########8 | 5.00M/5.66M [00:02<00:00, 2.46MB/s]
100%|##########| 5.66M/5.66M [00:02<00:00, 2.46MB/s]


In [134]:
import pandas as pd
import numpy as np

In [135]:
store = pd.read_csv("data/store.csv")

In [136]:
store.count()

Store                        1115
StoreType                    1115
Assortment                   1115
CompetitionDistance          1112
CompetitionOpenSinceMonth     761
CompetitionOpenSinceYear      761
Promo2                       1115
Promo2SinceWeek               571
Promo2SinceYear               571
PromoInterval                 571
dtype: int64

In [137]:
store['unknownCompetitionYear']=0
store.loc[store['CompetitionOpenSinceYear'].isnull(),'unknownCompetitionYear'] =1

In [138]:
store['CompetitionDistance'] = store['CompetitionDistance'].fillna(store['CompetitionDistance'].max())

In [139]:
store['CompetitionOpenSinceMonth'] = store['CompetitionOpenSinceMonth'].fillna(store['CompetitionOpenSinceMonth'].min())

In [140]:
cosy_fillna = store.groupby(['StoreType','Assortment'])['CompetitionOpenSinceYear'].min().reset_index()

In [141]:
cosy_fillna.fillna(2000,inplace=True)

In [142]:
cosy_fillna.loc[cosy_fillna['CompetitionOpenSinceYear']<1995,'CompetitionOpenSinceYear'] = 1995

In [143]:
#bb store opened way earlier compared to other  and not opened recently on average : 1999 - 2008 / 2002
#ba stores are relatively new : 2006 - 2014 / 2011
#bc store are not available

In [144]:
store = store.merge(cosy_fillna,left_on=['StoreType','Assortment'],right_on=['StoreType','Assortment'],suffixes=('','_fill'))

In [145]:
store['CompetitionOpenSinceYear'].fillna(store['CompetitionOpenSinceYear_fill'],inplace=True)

In [146]:
store.drop(columns=['CompetitionOpenSinceYear_fill'],axis=1,inplace=True)

In [147]:
store.isnull().any()

Store                        False
StoreType                    False
Assortment                   False
CompetitionDistance          False
CompetitionOpenSinceMonth    False
CompetitionOpenSinceYear     False
Promo2                       False
Promo2SinceWeek               True
Promo2SinceYear               True
PromoInterval                 True
unknownCompetitionYear       False
dtype: bool

In [148]:
df = pd.read_csv("data/train.csv")

C:\Users\Idea\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [150]:
df = df.merge(store,on='Store')

In [151]:
df['Date'] = pd.to_datetime(df['Date'])

In [159]:
df['StateHoliday'].unique()

array(['0', 'a', 'b', 'c', 0], dtype=object)

In [161]:
df.loc[df['StateHoliday']==0,'StateHoliday'] = '0'

In [163]:
df['isHoliday'] = (df['StateHoliday']!='0').astype(int)

In [173]:
df.groupby(['DayOfWeek'])['Sales'].describe()

,count,mean,std,min,25%,50%,75%,max
DayOfWeek,,,,,,,,
1,144730.0,7809.044510,4016.526273,0.0,5235.00,7310.0,9972.0,41551.0
2,145664.0,7005.244467,3142.039607,0.0,4904.00,6463.0,8491.0,34692.0
3,145665.0,6555.884138,2944.377501,0.0,4718.00,6133.0,7926.0,33151.0
4,145845.0,6247.575913,3209.806360,0.0,4536.00,6020.0,7792.0,38367.0
5,145845.0,6723.274305,3101.006008,0.0,4975.00,6434.0,8206.0,38722.0
6,144730.0,5847.562599,2873.985615,0.0,3899.25,5410.0,7220.0,31683.0
7,144730.0,204.183189,1613.243517,0.0,0.00,0.0,0.0,37376.0


In [174]:
df.groupby(['Store'])['Sales'].describe()

,count,mean,std,min,25%,50%,75%,max
Store,,,,,,,,
1,942.0,3945.704883,2015.417840,0.0,3588.25,4373.5,5220.00,9528.0
2,942.0,4122.991507,2363.628674,0.0,2806.75,4395.5,5804.50,10682.0
3,942.0,5741.253715,3298.782559,0.0,4247.50,5892.0,8152.75,15689.0
4,942.0,8021.769639,4012.592818,0.0,7322.50,9062.0,10281.00,17412.0
5,942.0,3867.110403,2389.609890,0.0,2027.00,4180.0,5419.25,11692.0
6,942.0,4562.375796,2527.359298,0.0,3408.75,5012.5,6333.50,11139.0
7,942.0,7356.902335,3969.299824,0.0,5947.25,7928.5,9785.75,18413.0
8,942.0,4610.251592,2696.883772,0.0,2916.75,4919.0,6558.00,10971.0
9,942.0,5426.816348,2891.687966,0.0,4765.50,5904.5,7086.75,13457.0


In [185]:
df.groupby(['Promo2'])['Sales'].describe()

,count,mean,std,min,25%,50%,75%,max
Promo2,,,,,,,,
0,508031.0,6124.710161,4136.143923,0.0,3901.0,6038.0,8283.0,38722.0
1,509178.0,5423.718218,3506.556202,0.0,3591.0,5478.0,7448.0,41551.0


In [ ]:
#finding out popular store 
#how many weeks/days the competetion store opened for
#PromoStartAnewmonth
#are they running promo before
#find possible clusters on store
#embedding on categorical features

In [175]:
df.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,unknownCompetitionYear,isHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,0,0
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,0,0
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,0,0
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,0,0
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN,0,0
